In [ ]:
!docker version

In [3]:
!mkdir flaskwebapp
!mkdir flaskwebapp/code
!mkdir flaskwebapp/etc

mkdir: cannot create directory ‘flaskwebapp’: File exists
mkdir: cannot create directory ‘flaskwebapp/code’: File exists
mkdir: cannot create directory ‘flaskwebapp/etc’: File exists


In [255]:
import os
from os import path
import json 

image_name = "thbeh/test"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [242]:
!ls -l flaskwebapp/code

total 314
-rwxrwxrwx 1 user00 user00    885 Nov 26 20:36 kill_supervisor.py
-rw-rw-rw- 1 user00 user00 159017 Nov 27 03:50 returnsModel_2.1-6.tar.gz
-rw-rw-rw- 1 user00 user00 160068 Nov 28 02:35 returnsModel_2.1-7.tar.gz
-rwxrwxrwx 1 user00 user00    377 Nov 27 05:47 runModel.R
-rw-rw-rw- 1 user00 user00    308 Nov 28 02:11 scoreDrill.R


In [241]:
!cp /poc/AZ/Kate/returnsModel_2.1-7.tar.gz flaskwebapp/code/

In [233]:
%%writefile flaskwebapp/packages.R
install.packages(c("psych"
                  ), 
                   repos = "http://cran.us.r-project.org", dependencies = TRUE)

## install.packages("/poc/AZ/Kate/returnsModel_2.1-6.tar.gz", repos = NULL, type = "source")


Overwriting flaskwebapp/packages.R


In [272]:
%%writefile flaskwebapp/runModel.R

suppressMessages(library(psych))
suppressMessages(library(jsonlite))
suppressMessages(library(sergeant))
suppressMessages(library(returnsModel))

options(warn = -1)

#* @get /testScore
runScore <- function() {
    RunModellingAndScoringTest(model="OR", returnName="IR6", projectStage="TEST", nrows=1000)
}

#* @post /score
runModel <- function(nr) {
    RunModellingAndScoringTest(model="OR", returnName="IR6", projectStage="TEST", nrows=nr)
}

#* @post /sum
addTwo <- function(a ,b) {
    as.numeric(a) + as.numeric(b)
}



Overwriting flaskwebapp/runModel.R


In [258]:
%%writefile flaskwebapp/supervisord.conf
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:plumber]
command=/usr/bin/Rscript -e "pr <- plumber::plumb('/tmp/runModel.R'); pr$run(host='0.0.0.0', port=8080)"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2





Writing flaskwebapp/supervisord.conf


In [259]:
%%writefile flaskwebapp/dockerfile
FROM thbeh/r-base:3.4.2

RUN apt-get update && apt-get install -y supervisor libxml2-dev

COPY packages.R /tmp/
COPY runModel.R /tmp/
COPY scoreDrill.R /tmp/
COPY supervisord.conf /etc/

RUN mkdir /code
ADD code /code

RUN Rscript /tmp/packages.R
RUN R CMD INSTALL /code/returnsModel_2.1-7.tar.gz

EXPOSE 8080
CMD ["supervisord","-c", "/etc/supervisord.conf"]



Overwriting flaskwebapp/dockerfile


In [237]:
! ls -l flaskwebapp/code

total 314
-rwxrwxrwx 1 user00 user00    885 Nov 26 20:36 kill_supervisor.py
-rw-r--r-- 1 user00 user00 159017 Nov 27 03:50 returnsModel_2.1-6.tar.gz
-rw-r--r-- 1 user00 user00 160068 Nov 28 00:23 returnsModel_2.1-7.tar.gz
-rwxrwxrwx 1 user00 user00    377 Nov 27 05:47 runModel.R
-rw-r--r-- 1 user00 user00    381 Nov 27 07:02 scoreDrill.R


In [275]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon 262.1 MB
Step 1/12 : FROM thbeh/r-base:3.4.2
 ---> e86845d98544
Step 2/12 : RUN apt-get update && apt-get install -y supervisor libxml2-dev
 ---> Running in 73d1bdd64af6
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]
Ign:2 http://package.mapr.com/releases/v5.2.2/ubuntu binary InRelease
Get:3 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Ign:4 http://package.mapr.com/releases/MEP/MEP-3.0.1/ubuntu binary InRelease
Get:5 http://package.mapr.com/releases/v5.2.2/ubuntu binary Release [2,728 B]
Get:6 http://package.mapr.com/releases/MEP/MEP-3.0.1/ubuntu binary Release [2,728 B]
Get:7 http://package.mapr.com/releases/v5.2.2/ubuntu binary Release.gpg [475 B]
Get:8 http://package.mapr.com/releases/MEP/MEP-3.0.1/ubuntu binary Release.gpg [475 B]
Get:9 http://package.mapr.com/releases/v5.2.2/ubuntu binary/trusty amd64 Packages [3,249 B]
Get:10 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [53.1 kB

In [ ]:
!docker run -it -e MAPR_CLUSTER=maprlab -e MAPR_CLDB_HOSTS=mapr-1,mapr-2,mapr-3 -e MAPR_CONTAINER_USER=mapr -e MAPR_CONTAINER_UID=5000 -e MAPR_CONTAINER_GID=5000 -e MAPR_CONTAINER_GROUP=mapr -e MAPR_MOUNT_POINT=/mapr --device /dev/fuse --security-opt apparmor:unconfined --name score -p 6060:8080 $image_name 

[!p>
Testing for cluster user account... 

(BEnter MapR cluster user name: mapr
 
User 'mapr' does not exist. Creating new cluster user account... 

Enter 'mapr' uid: 5000
 Enter 'mapr' group name: mapr
 Enter 'mapr' gid: 5000
 Enter 'mapr' password: mapr
 ...Success 

Configuring MapR client ( -c -C mapr-1,mapr-2,mapr-3 -N maprlab)... 

create /opt/mapr/conf/conf.old
Configuring Hadoop-2.7.0 at /opt/mapr/hadoop/hadoop-2.7.0
Done configuring Hadoop
CLDB node list: mapr-1:7222,mapr-2:7222,mapr-3:7222
Zookeeper node list: 

...Success 

2017-11-29 02:14:45,944 INFO supervisord started with pid 946
2017-11-29 02:14:46,947 INFO spawned: 'program_exit' with pid 949
2017-11-29 02:14:46,948 INFO spawned: 'plumber' with pid 950
2017-11-29 02:14:48,942 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2017-11-29 02:14:52,947 INFO success: plumber entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)


In [274]:
!docker rm score

Error response from daemon: No such container: score


In [178]:
!docker rmi 0dbdaaafb97c


Deleted: sha256:0dbdaaafb97cfd2347d7ea792bc75ec14efbfa6994baa71d671b726ec4f3f034
Deleted: sha256:aa0ebefcb4ba7d8c8efea6e9a5da852cc05f927b606649c6e64aa797c3849eb1
Deleted: sha256:ef510ba0f80c2fcf7db635528b2edacf6646d5540fd6d62d07968ff90328f06e
Deleted: sha256:d7050fd3e87eade8ad3051b5a910145fa97baabe574c1486437d471b444cfc54
Deleted: sha256:35c44dc6eacd611c412786c9b319c82dc779fd8616e769fd2c759ac09213ae6c
Deleted: sha256:4499cc5cea262e03b6ffa8ccb948c4398d258c8176eec84734a88c4d69afbafb
Deleted: sha256:8d7db60bf1198940faae1e8c193518cc7bf971a63b70f63296dcd87514374d01
Deleted: sha256:8ec04630f111f16df53e32a0d2e21d19972c51e233df72d845720f0cf2f637d5
Deleted: sha256:abf39fd24dc3324cf0dd3a0363bec75f77d787c4d9f405c0e8e766e487543f32
Deleted: sha256:c644ae04a78eec19b227b03c24959fb9ca18af07c8a4a0f3d75f7ee310671882
Deleted: sha256:b34207502e7f4af737b373d06e61d8c6f70a3788142f308a768130ba7436dbd8
Deleted: sha256:a41925a6e229204b61c0c42d76122c18aed9f7402f7690f32322f89574d732a2


In [221]:
!docker rm score


score


In [215]:
!docker rmi thbeh/test

Untagged: thbeh/test:latest
Deleted: sha256:523fa3410c68f037c5a3140cf12f47244aeffa3ed6139c508eef04193c6de130
Deleted: sha256:fed7840dfe6bca268b4f65f8f200f272c3b061df8efcdf415b1ca364302c3550
Deleted: sha256:d182afd54e90ca32fdfd5085f481c5fd3dfe79fa20d70fbf28a3a5f9950114b8
Deleted: sha256:07e0e896719c1a9e67eb16341d8ffe1fd538c9cd1ac7da18c92ef0846b1103d3
Deleted: sha256:cbdd436ff8a553e6b6cd713102aa1f2106255770c6ea69d7945d8b9547c07fe9
Deleted: sha256:668ab1eeea96c2f9a74a65fe997d3a9c864da3eb2d46d0b696b2293813d16838
Deleted: sha256:83e02988213471dde80a1d0694d6beef615a74ab300d8bedf3f429e5ddb4082c
Deleted: sha256:73f64dc02f96c7a9515d9dc94d6e0afa1ac9c0d2639b3522f70fe484857cf944
Deleted: sha256:e111832e869c91ee4179747f47bb3744150b6dc7c2aa436a043020e06e1be405
Deleted: sha256:ca64bfb0614195eea824868d9fed85b0a36614506af607d28050eb5785e7c4ca
Deleted: sha256:5ebaead3cc58368dfee19eb91e454ebae2e946f19246037d7d0abe6334bdf7ac
Deleted: sha256:9298ea1c34471b00fcd95e0722ffe9a525d7c5cb0aac39c1db93a4cb04bfac4d
